In [1]:
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.simulations.projectile import Projectile
from autoemulate.experimental.calibration.history_matching import HistoryMatching

# History Matching

In this tutorial we demonstrate the use of History Matching to determine which points in the input space are plausible given a set of observations. 

Performing History Matching requires:
- a fit emulator that predicts uncertainty (e.g., Gaussian Process) and 
- an observation associated with the simulator output. 

The emulator is used to efficiently estimate the simulator output, accounting for all uncertainties. The emulated output is then compared with the observation and parameter inputs that are unlikely to produce the observation can then be “ruled out” as implausible, reducing the input space.

## 1. Simulate data and train an emulator

In this example we use the `Projectile` simulator. It returns distance travelled by a projectile given a drag coefficient `c` and initial velocity `v0`.

In [2]:
simulator = Projectile()
x = simulator.sample_inputs(1000)
y = simulator.forward_batch(x)

Running simulations: 100%|██████████| 1.00k/1.00k [00:00<00:00, 1.44ksample/s]


Since we need an uncertainty aware emulator, we restrict AutoEmulate to only train a Gaussian Process.

In [3]:
ae = AutoEmulate(x, y, models=[GaussianProcessExact])

Comparing models: 100%|██████████| 1.00/1.00 [00:20<00:00, 20.3s/model]


We can verify that the fitted emulator performs well on both the train and test data.

In [4]:
ae.summarise()

,model_name,x_transforms,y_transforms,rmse_score,r2_score
0,GaussianProcessExact,[StandardizeTransform()],[StandardizeTransform()],2146613.0,0.962294


In [5]:
model = ae.best_result().model

## 2. Calibrate

To instantiate the `HistoryMatching` object, we need an observed mean and, optionally, variance for each simulator output. 

In [6]:
#  observed data: (mean, variance)
hm = HistoryMatching(observations={"distance": (2500, 100)})

We also need predictions for a set of query points using the trained emulator. These must have uncertainty estimates.

In [7]:
x_new = simulator.sample_inputs(10)
pred = model.predict(x_new)

Primary use of the `HistoryMatching` class is the `calculate_implausibility` method, which returns the implausibility metric (a number of standard deviations between the emulator mean and the observation) for the queried points.

In [8]:
implausability = hm.calculate_implausibility(pred.mean, pred.variance)
implausability

tensor([[ 2.5518],
        [ 4.1512],
        [ 9.6116],
        [ 2.0852],
        [ 0.6714],
        [ 1.1179],
        [ 1.1520],
        [ 2.0755],
        [16.3014],
        [ 1.6649]])

The `get_nroy` method can be used to filter the queried points given the implausability scores and only retain those that have not been ruled out yet (NROY).

In [9]:
hm.get_nroy(implausability, x_new)

tensor([[-2.3097e+00,  8.7343e+02],
        [-1.4562e-01,  7.9759e+02],
        [-3.1836e+00,  2.2645e+02],
        [-4.1329e-01,  6.2724e+02],
        [-3.6554e+00,  3.0588e+02],
        [-1.0273e+00,  6.2846e+01],
        [ 7.5454e-01,  1.9359e+02]])